In [15]:
import pandas as pd
import numpy as np
import os

In [ ]:
input_file_name = 'data-v2.xlsx'
file_path = os.path.join(os.getcwd(), 'input-files', input_file_name)
read_columns = ['Time_stamp', 'SOLAR_GHI_GROUND_01_AVG', 'SOLAR_DIRECTRADIATION_GROUND_01_AVG', 'SOLAR_DIFFUSERADIATION_GROUND_01_AVG', 'DAY_OF_YEAR', 'DECLINATION', 'EOT', 'TC', 'LST', 'HOUR_ANGLE', 'ZENITH_ANGLE']
data_DF = pd.read_excel(file_path, usecols=read_columns)
data_DF.head()

In [ ]:
# data_DF.head()
# data_DF.dtypes
# data_DF.isnull().sum()
if data_DF.isnull().values.any():
    data_DF = data_DF.dropna()
# data_DF.duplicated().sum()
# data_DF.describe()

In [ ]:
# Calculation for I0n
# Extraterrestrial irradiance on a normal surface 
# I0n = Isc(1 + 0.033 * cos( 360 * n/ 365))
# Isc​ is the solar constant (1367 W/m²)
# n is the day of the year (from the date in your period column)

data_DF['I0n'] = 1367 * (1 + 0.033 * np.cos(2 * np.pi * data_DF['DAY_OF_YEAR'] / 365))
data_DF['COS_ZENITH'] = np.cos(data_DF['ZENITH_ANGLE'])
data_DF.rename(columns={
    'SOLAR_GHI_GROUND_01_AVG': 'GHI', 
    'SOLAR_DIRECTRADIATION_GROUND_01_AVG': 'DNI', 
    'SOLAR_DIFFUSERADIATION_GROUND_01_AVG': 'DHI'
}, inplace=True)
data_DF['Kt'] = data_DF['GHI'] / (data_DF['I0n'] * data_DF['COS_ZENITH'])
data_DF['Kd'] = data_DF['DHI'] / data_DF['GHI']
data_DF['Kn'] = data_DF['DNI'] / data_DF['I0n']

# # Horizontal extraterrestrial irradiance G0h
data_DF['G0h'] = data_DF['I0n'] * data_DF['COS_ZENITH']
# data_DF.head()
data_DF.describe()

c:\Users\zeesh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h
count,43843,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,43843.000000,2.276500e+04,43843.000000,43843.000000
mean,2012-03-20 18:13:28.521314816,96.483501,138.949679,191.801936,167.135483,-0.055212,-1.281610,306.823766,17.039845,75.597669,90.091261,1368.608011,-0.003933,-0.001325,inf,0.101521,-5.360337
min,2011-10-20 12:20:00,0.000000,0.000000,0.000000,1.000000,-23.449783,-14.258212,293.847164,4.897453,-106.538209,5.064979,1321.890671,-1.000000,-1505.995226,0.000000e+00,0.000000,-1412.076387
25%,2012-01-04 15:35:00,0.000000,0.000000,0.000000,77.000000,-18.547676,-6.584533,301.520843,11.036574,-14.451386,55.346919,1331.438771,-0.710733,-0.002308,4.239986e-01,0.000000,-971.957997
50%,2012-03-20 18:20:00,1.306513,0.222270,1.407574,153.000000,-0.403653,-2.515903,305.589473,17.047711,75.715658,90.079745,1375.681683,-0.008046,0.000000,6.111101e-01,0.000161,-10.708078
75%,2012-06-04 21:05:00,188.241005,265.771805,364.054611,229.000000,18.547676,3.559073,311.664449,23.038367,165.575500,124.835725,1402.798809,0.703105,0.001451,9.043194e-01,0.194240,961.706447
max,2012-08-19 23:50:00,645.252136,869.604736,1069.615234,365.000000,23.449783,16.380523,324.485899,29.241432,258.621475,174.992079,1412.111000,1.000000,720.119764,inf,0.629050,1411.968550
std,NaN,127.774209,213.271242,271.433163,108.308637,17.894359,8.643346,8.643346,6.930213,103.953191,41.726563,34.214096,0.706387,10.868303,NaN,0.155693,966.936728


In [ ]:
# K-tests
k_test_condition_1 = (data_DF['GHI'] > 50) & (data_DF['ZENITH_ANGLE'] < 75) & (data_DF['Kd'] < 1.05)
k_test_condition_2 = (data_DF['GHI'] > 50) & (data_DF['ZENITH_ANGLE'] > 75) & (data_DF['Kd'] < 1.10)
k_test_condition_3 = (data_DF['Kn'] < 0.8) & (data_DF['Kn'] > 0)
k_test_condition_4 = (data_DF['Kd'] < 0.96) & (data_DF['Kt'] > 0.6)

data_DF['k_test'] = np.where(
    k_test_condition_1 | k_test_condition_2 | k_test_condition_3 | k_test_condition_4,
    'Passed',
    'Failed'
)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.509843,323.615219,17.726920,85.903805,91.871027,1381.676507,-0.721493,-0.654534,0.750141,0.301014,-996.869687,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.509843,323.615219,17.893587,88.403805,94.025719,1381.676507,0.975446,0.429479,0.976274,0.309041,1347.750246,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.509843,323.615219,18.060254,90.903805,96.188764,1381.676507,-0.361791,-0.985761,0.973463,0.237218,-499.878384,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.509843,323.615219,18.226920,93.403805,98.359209,1381.676507,-0.565435,-0.608918,0.517614,0.221697,-781.247672,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.509843,323.615219,18.393587,95.903805,100.536133,1381.676507,0.999987,0.349319,0.854338,0.065544,1381.658058,Passed


In [ ]:
#  Individual Limits Test (ILT)
ILT_condition_1 = (data_DF['GHI'] > -4) & (data_DF['GHI'] < (1.5 * data_DF['I0n'] * data_DF['COS_ZENITH']**1.2 + 100))
ILT_condition_2 = (data_DF['DHI'] > -4) & (data_DF['DHI'] < (0.97 * data_DF['I0n'] * data_DF['COS_ZENITH']**1.2 + 50))
ILT_condition_3 = (data_DF['DNI'] > -4) & (data_DF['DNI'] < data_DF['I0n'])
ILT_condition_4 = (data_DF['DHI'] < 0.8 * data_DF['G0h'])
ILT_condition_5 = (data_DF['GHI'] - data_DF['DHI'] < data_DF['G0h'])

data_DF['individual_limit_test'] = np.where(
    ILT_condition_1 | ILT_condition_2 | ILT_condition_3 | ILT_condition_4 | ILT_condition_5,
    'Passed',
    'Failed'
)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test,individual_limit_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.509843,323.615219,17.726920,85.903805,91.871027,1381.676507,-0.721493,-0.654534,0.750141,0.301014,-996.869687,Passed,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.509843,323.615219,17.893587,88.403805,94.025719,1381.676507,0.975446,0.429479,0.976274,0.309041,1347.750246,Passed,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.509843,323.615219,18.060254,90.903805,96.188764,1381.676507,-0.361791,-0.985761,0.973463,0.237218,-499.878384,Passed,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.509843,323.615219,18.226920,93.403805,98.359209,1381.676507,-0.565435,-0.608918,0.517614,0.221697,-781.247672,Passed,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.509843,323.615219,18.393587,95.903805,100.536133,1381.676507,0.999987,0.349319,0.854338,0.065544,1381.658058,Passed,Passed


In [ ]:
# Night-Time Zero Testing
filter_condition_1 = data_DF['GHI'] > 5
filter_condition_2 = data_DF['ZENITH_ANGLE'] < 85
data_DF['night_time_test'] = np.where(
    filter_condition_1 | filter_condition_2,
    'Passed',
    'Failed'
)
output_file_name = 'output-v2.csv'
output_path = os.path.join(os.getcwd(), 'output-files', output_file_name)
data_DF.to_csv(output_path, encoding='utf-8', index=None)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test,individual_limit_test,night_time_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.509843,323.615219,17.726920,85.903805,91.871027,1381.676507,-0.721493,-0.654534,0.750141,0.301014,-996.869687,Passed,Passed,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.509843,323.615219,17.893587,88.403805,94.025719,1381.676507,0.975446,0.429479,0.976274,0.309041,1347.750246,Passed,Passed,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.509843,323.615219,18.060254,90.903805,96.188764,1381.676507,-0.361791,-0.985761,0.973463,0.237218,-499.878384,Passed,Passed,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.509843,323.615219,18.226920,93.403805,98.359209,1381.676507,-0.565435,-0.608918,0.517614,0.221697,-781.247672,Passed,Passed,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.509843,323.615219,18.393587,95.903805,100.536133,1381.676507,0.999987,0.349319,0.854338,0.065544,1381.658058,Passed,Passed,Passed
